In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
data=pd.read_csv("data (5).csv")
data.head()


In [ ]:
data.isnull().sum()

In [ ]:
data=data.drop(["id","Unnamed: 32"],axis=1)
data.head()

In [ ]:
plt.figure(figsize = (20,20))
sns.heatmap(data[data.columns].corr(), annot = True, fmt = ".2f")
plt.show()

In [ ]:
data.diagnosis=[1 if i=="B" else 0 for i in data.diagnosis]
y=data.diagnosis.values
x_data=data.drop(["diagnosis"],axis=1)
#normalization
x=(x_data-np.min(x_data))/(np.max(x_data)-np.min(x_data))
m=data[data.diagnosis == 0]
b=data[data.diagnosis == 1]
b.head()

In [ ]:
colons_name=["radius_mean","texture_mean","perimeter_mean","smoothness_mean","compactness_mean","fractal_dimension_worst","symmetry_worst","symmetry_worst"]
for i in colons_name:
    for j in colons_name:
        plt.figure(figsize = (5,5))
        plt.scatter(m[i],m[j],color="red",label="kötü huylu")
        plt.scatter(b[i],b[j],color="green",label="iyi huylu")
        plt.xlabel(i)
        plt.ylabel(j)
        plt.show()


In [ ]:
colons_name=["radius_mean","texture_mean","perimeter_mean","smoothness_mean","compactness_mean","fractal_dimension_worst","symmetry_worst","symmetry_worst"]
for i in colons_name:
    plt.figure(figsize = (5,5))
    data.boxplot(column=i,by = "diagnosis")
    plt.xlabel("diagnosis")
    plt.ylabel(i)
    plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.3,random_state=42)


def knn_score_distance(neighbors,distance):
    knn=KNeighborsClassifier(n_neighbors=neighbors)
    knn.fit(x_train,y_train)
    prediciton=knn.predict(x_test)
    return float("{:.2f}".format(knn.score(x_test,y_test)*1000))


metrics=["euclidean","manhattan","chebyshev","minkowski"]
score=[]
for i in range(1,4):
    for j in metrics:
        score.append(knn_score_distance(i,j))
        
    plt.figure(figsize = (12,9))
    plt.barh(metrics, score)
    for index, value in enumerate(score):
        plt.text(value, index, str(value))
 
    plt.xlabel("knn-score-same-n")
    plt.ylabel("distance-metric")
    plt.title("neighbors:"+str(i))
    plt.show()


In [ ]:
score=[]
for i in range(1,12):
    score.append(knn_score_distance(i,"manhattan"))


plt.plot(range(1,12),score)
plt.xlabel("negihbor-number")
plt.ylabel("knn-score")
plt.title("manhattan-distance-score-for-every-neighbors-number")
plt.show()

In [ ]:
knn=KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train,y_train)
prediciton=knn.predict(x_test)
true=y_test

from sklearn.metrics import confusion_matrix

cm=confusion_matrix(true,prediciton)
sns.heatmap(cm/np.sum(cm), annot=True, 
            fmt='.2%', cmap='Blues')
plt.show()

print(cm)

In [ ]:
#random forest
from sklearn.ensemble import RandomForestClassifier

def random_forest_score_forest_number(forest_number):
    rf=RandomForestClassifier(n_estimators=forest_number,random_state=42)
    rf.fit(x_train,y_train)
    return rf.score(x_test,y_test)
    
score=[]
for i in range(20,120,10):
    score.append(random_forest_score_forest_number(i))
    
plt.plot(range(20,120,10),score)
plt.xlabel("tree-number")
plt.ylabel("randomforest-score")
plt.title("random-forest-score-for every-tree-number")
plt.show()    

In [ ]:
rf=RandomForestClassifier(n_estimators=40,random_state=42)
rf.fit(x_train,y_train)
prediciton=rf.predict(x_test)
true=y_test

cm=confusion_matrix(true,prediciton)
sns.heatmap(cm/np.sum(cm), annot=True, 
            fmt='.2%', cmap='Blues')
plt.show()

print(cm)

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential # initialize neural network library
from keras.layers import Dense # build our layers library
def build_classifier():
    classifier = Sequential() # initialize neural network
    classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu', input_dim = x_train.shape[1]))
    classifier.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
def ann(epoch):
    classifier = KerasClassifier(build_fn = build_classifier, epochs = epoch)
    accuracies = cross_val_score(estimator = classifier, X = x_train, y = y_train)
    return accuracies
for i in range(50,350,50):
    accuracies=ann(i)
    print("Accuracy: "+ str(accuracies))

In [ ]:
score=[0.9707,0.9707,0.9875]
algorithm=["KNN","Random Forest","ANN"]

plt.plot(algorithm,score)
plt.xlabel("algorithm")
plt.ylabel("best-score")
plt.title("score-for every-algorithm-with-outliers")
plt.show()

In [ ]:
score=[0.9707,0.9597,0.9852]
algorithm=["KNN","Random Forest","ANN"]

plt.plot(algorithm,score)
plt.xlabel("algorithm")
plt.ylabel("best-score")
plt.title("score-for every-algorithm-without-outliers")
plt.show()